In [27]:
#! /usr/bin/env python

import numpy as np
import cv2
import sys
from PIL import ImageGrab
import pyautogui
import os
import time



In [33]:
source = 0  # defaults to local (laptop) camera


cap = cv2.VideoCapture(source)  # Start the video source
cap.set(3,1280)
cap.set(4,1024)
ret, ref_img = cap.read()         # readackground image
#count=1


#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)

os.chdir(r"C:\Users\bharath\Desktop\aa\captured")
#if (ret == False):
#    print("Unable to get any images.  Is the camera already in use?")
#    sys.exit()


# setup for reading & writing movie files
if (source !=0):                   
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    video_out = cv2.VideoWriter()
    frame_shape = (ref_img.shape[1],ref_img.shape[0])
    success = video_out.open('greenscreen_movie_post.mp4v',fourcc, 15.0, frame_shape,True)


# initializing background arrays
green_img = np.zeros(ref_img.shape, np.uint8)
fgmask = np.zeros(ref_img.shape, np.uint8)
green_img[:] = (0, 255, 0)
orig_ref = ref_img
orig_green = green_img
result = ref_img 



# this is for automatically setting corners of frame to background
gray_shape = (ref_img.shape[0],ref_img.shape[1])
bg_edges = np.zeros(gray_shape, np.uint8)
# storage format is y,x, color
edge_size_x = int(bg_edges.shape[1]/8)
edge_size_y = int(bg_edges.shape[0]/4)
bg_edges[0:edge_size_y,0:edge_size_x] = 255
bg_edges[0:edge_size_y,-edge_size_x:] = 255
bg_edges[-edge_size_y:,0:edge_size_x] = 255
bg_edges[-edge_size_y:,-edge_size_x:] = 255


avg1 = np.float32(ref_img)   # time-averaging array


def find_fgmask(img,ref_img,thresh=13.0,use_denoise=False,h=10.0):
    diff1 = cv2.subtract(img,ref_img)
    diff2 = cv2.subtract(ref_img,img)
    diff = diff1+diff2 
    
    sws = int(   np.ceil(21*h/10) // 2 * 2 + 1 )
    diff[ abs(diff) < thresh] = 0
    gray = cv2.cvtColor(diff.astype(np.uint8), cv2.COLOR_BGR2GRAY)
    gray[np.abs(gray) < 10] = 0
    if (use_denoise):
        cv2.fastNlMeansDenoising(gray,gray,h=h,templateWindowSize=5,searchWindowSize=sws)
    fgmask = gray.astype(np.uint8)
    fgmask[ fgmask >0] = 255
    return fgmask 



# Cropping stuff
refPt = []
setting_cropping = False
def click_and_crop(event, x, y, flags, param):
    # grab references to the global variables
    global refPt, cropping
    # if the left mouse button was clicked, record the starting
    # (x, y) coordinates and indicate that cropping specification is being
    # performed
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        setting_cropping = True
    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        # record the ending (x, y) coordinates and indicate that
        # the cropping operation is finished
        refPt.append((x, y))
        setting_cropping = False
        # draw a rectangle around the region of interest
        cv2.rectangle(result, refPt[0], refPt[1], (255, 255, 0), 2)
        cv2.imshow("frame", result)


cv2.namedWindow("frame")
cv2.setMouseCallback("frame", click_and_crop)  # register mouse events


print("Controls:")
print("D = difference image: take reference image & subtract")
print("C = clear settings: clear reference image, denoising")
print("T = toggle time averaging (probably don't want to use this)")
print("K = toggle crop (Draw box with mouse first, then press K)")
print("N = toggle de-noising (slow)")
print("Up/Down arrows = threshold for foreground: more/less green")
print("Left/Right arrows = size of denoising kernel: less/more noise")
print("Q = quit")
print("P= Screenshot")

# control parameters
use_diff = True 
use_time_avg = False
use_denoise = False
denoise_h = 10.0
use_cropping = False
thresh = 13.0
already_pressed_clear = False
use_edges = False

img_counter = 0
# Main Loop
while(1):
    #cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("frame", 1920,1080)
    ret, img = cap.read()
    if (False==ret):   # end of video capture
        print("Saving and terminating")
        break

    # crop first, for speed
    if (use_cropping) and (len(refPt)==2):
        cropped_img = img[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]]
        img = cropped_img
        if (img.shape[0] < ref_img.shape[0]) and (img.shape[1] < ref_img.shape[1]):
            cropped = ref_img[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]]
            ref_img = cropped
        if (img.shape[0] < green_img.shape[0]) and (img.shape[1] < green_img.shape[1]):
            cropped = green_img[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]]
            green_img = cropped
        if (img.shape[0] < avg1.shape[0]) and (img.shape[1] < avg1.shape[1]):
            avg1 = np.float32(ref_img)
    
    result = img 
    if (use_time_avg):
        cv2.accumulateWeighted(img,avg1,0.09)
        img = cv2.convertScaleAbs(avg1)
    if (use_diff):
        fgmask = find_fgmask(img,ref_img,thresh=thresh,use_denoise=use_denoise,h=denoise_h)
        bgmask = cv2.bitwise_not(fgmask)
        if (use_edges):
            bgmask = cv2.bitwise_or(bgmask,bg_edges)
            fgmask = cv2.bitwise_not(bgmask)
        fgimg = cv2.bitwise_and(img,img,mask = fgmask)
        bgimg = cv2.bitwise_and(green_img,green_img,mask = bgmask)
        sum = cv2.add(fgimg, bgimg)
        result = sum
    # if there's a cropping rectangle drawn, keep showing the rectangle
    if ((2==len(refPt)) and (not use_cropping)):
        cv2.rectangle(result, refPt[0], refPt[1], (255, 255, 0), 2)
    cv2.imshow('frame',sum)
    if (0!=source):
        video_out.write(result)
    # if there's a cropping rectangle drawn, keep showing the rectangle
    if ((2==len(refPt)) and (not use_cropping)):
        cv2.rectangle(result, refPt[0], refPt[1], (255, 255, 0), 2)
    key = cv2.waitKey(1) & 0xFF
    if ord('q') == key:
        break
    elif ord('d') == key:      # take & use difference image
        use_diff = True
        ref_img = img
    elif ord('e') == key:    # add extra green to corners
        use_edges = not use_edges
    elif key == ord('t'):    # toggle time-averaging
        use_time_avg = not use_time_avg
        avg1 = np.float32(img)
    elif key == ord('c'): # clear button
        print("already_pressed_clear = ",already_pressed_clear)
        if (already_pressed_clear):
            use_diff = use_diff_save
            use_time_avg = use_time_avg_save
            use_denoise = use_denoise_save
            denoise_h = denoise_h_save
            thresh = thresh_save
            already_pressed_clear = False 
        else:
            use_diff_save = use_diff
            use_denoise_save = use_denoise
            use_time_avg_save = use_time_avg
            denoise_h_save = denoise_h
            thresh_save = thresh
            already_pressed_clear = True
            use_diff = False 
            use_time_avg = False
            use_denoise = False
            denoise_h = 10.0
            thresh = 13.0
    elif key == ord('n'): # toggle denoising (slow)
        use_denoise = not use_denoise
        if (use_denoise):
            denoise_h = 10.0
        else:
            denoise_h = 0.0
    elif (key == ord('k')) and (2==len(refPt)):  # cropping
        use_cropping = not use_cropping
        if (not use_cropping):
            ref_img = orig_ref
            green_img = orig_green
            avg1 = np.float32(ref_img)	 
    elif (key == 0):  # up arrow
        thresh *= 1.1     # increase difference threshold = more green
    elif (key == 1):  # down arrow
        thresh /= 1.1
    elif (key == 2):  # left arrow
        denoise_h /= 1.1  # increase both denoising weight and kernel 
    elif (key == 3):  # right arrow
        denoise_h *= 1.1
    elif key == ord('p'):
        
        img_name = "opencv_frame_{}_{}.png".format(time.strftime("%Y%m%d-%H%M%S"), img_counter)
        cv2.imwrite(img_name, result)
        print("{} written!".format(img_name))
        img_counter += 1
        #image = pyautogui.screenshot()
        #image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        #cv2.imwrite("capture"+str(count)+".png", image)
        #count=count+1
# Main Loop has finished, shutting down

if source != 0:        # close & release any output file
    video_out.release()
    video_out = None 

cap.release()       # release the camera
cv2.destroyAllWindows()

Controls:
D = difference image: take reference image & subtract
C = clear settings: clear reference image, denoising
T = toggle time averaging (probably don't want to use this)
K = toggle crop (Draw box with mouse first, then press K)
N = toggle de-noising (slow)
Up/Down arrows = threshold for foreground: more/less green
Left/Right arrows = size of denoising kernel: less/more noise
Q = quit
P= Screenshot
opencv_frame_20190318-173038_0.png written!


In [31]:
cap.release()       # release the camera
cv2.destroyAllWindows()

In [ ]:
refPt